<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import glob

def create_image_pyramid(image_path, levels):
    image = cv2.imread(image_path)
    pyramid = [image]
    for _ in range(levels):
        image = cv2.pyrDown(image)
        pyramid.append(image)
    return pyramid

def create_voxel_data(image_paths, levels):
    voxel_data_list = []
    for image_path in image_paths:
        pyramid = create_image_pyramid(image_path, levels)
        high_res_image = pyramid[0]
        voxel_data = np.zeros((high_res_image.shape[0], high_res_image.shape[1], 3), dtype=np.uint8)
        for i in range(high_res_image.shape[0]):
            for j in range(high_res_image.shape[1]):
                voxel_data[i, j] = high_res_image[i, j]
        voxel_data_list.append(voxel_data)
    return voxel_data_list

def display_voxel_data(voxel_data_list):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    for voxel_data in voxel_data_list:
        x, y, z = voxel_data.nonzero()
        ax.scatter(x, y, z, c=voxel_data[x, y, z] / 255.0, marker='o')
    plt.show()

# メイン処理
image_paths = glob.glob('path_to_your_images/*.tif')  # 画像ファイルのパスを指定
voxel_data_list = create_voxel_data(image_paths, 3)
display_voxel_data(voxel_data_list)

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Voxel 3D with Ray Casting and Image Pyramid</title>
    <style>
        body { margin: 0; overflow: hidden; }
        canvas { display: block; }
    </style>
</head>
<body>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
    <script>

        let scene, camera, renderer, controls;
        let voxelGroup = new THREE.Group();
        const imagePyramids = [];
        const pyramidLevels = 4;

        // ピラミッド画像処理を用いて異なる解像度の画像を事前にロード
        function createImagePyramid(baseImageUrl, levels) {
            const textures = [];
            for (let i = 0; i < levels; i++) {
                let scale = Math.pow(2, i);
                let url = `${baseImageUrl}_${scale}.tif`; // 低解像度バージョンをサーバーに用意しておく
                let texture = new THREE.TextureLoader().load(url);
                textures.push(texture);
            }
            return textures;
        }

        // ピラミッド画像の解像度選択 (ズームレベルに応じてテクスチャを変更)
        function getTextureForZoom(zoomLevel, pyramidTextures) {
            const level = Math.min(pyramidLevels - 1, Math.max(0, Math.floor(zoomLevel)));
            return pyramidTextures[level];
        }

        function init() {
            // シーンとカメラ、レンダラの設定
            scene = new THREE.Scene();
            camera = new THREE.PerspectiveCamera(75, window.innerWidth / window.innerHeight, 0.1, 1000);
            camera.position.z = 500;

            renderer = new THREE.WebGLRenderer();
            renderer.setSize(window.innerWidth, window.innerHeight);
            document.body.appendChild(renderer.domElement);

            // 500枚の画像を使ってボクセルを構築
            for (let i = 0; i < 500; i++) {
                const pyramidTextures = createImagePyramid(`image_${i}`, pyramidLevels);
                imagePyramids.push(pyramidTextures);

                // 3Dの各ボクセルを定義（画像をBoxGeometryに貼り付ける）
                const geometry = new THREE.BoxGeometry(1, 1, 1);
                const material = new THREE.MeshBasicMaterial({
                    map: getTextureForZoom(camera.position.z / 500, pyramidTextures) // ズームに基づいてテクスチャを選択
                });
                const voxel = new THREE.Mesh(geometry, material);
                voxel.position.set(Math.random() * 500 - 250, Math.random() * 500 - 250, Math.random() * 500 - 250);
                voxelGroup.add(voxel);
            }

            scene.add(voxelGroup);

            // レイキャストによるインタラクション処理
            document.addEventListener('mousemove', onDocumentMouseMove, false);
            window.addEventListener('resize', onWindowResize, false);

            animate();
        }

        // ウィンドウサイズ変更時の処理
        function onWindowResize() {
            camera.aspect = window.innerWidth / window.innerHeight;
            camera.updateProjectionMatrix();
            renderer.setSize(window.innerWidth, window.innerHeight);
        }

        // マウス移動時の処理
        function onDocumentMouseMove(event) {
            // レイキャストで対象のボクセルに応じた処理を行う
        }

        // レンダリングループ
        function animate() {
            requestAnimationFrame(animate);

            // ズームレベルに応じたテクスチャ変更
            voxelGroup.children.forEach((voxel, index) => {
                const pyramidTextures = imagePyramids[index];
                voxel.material.map = getTextureForZoom(camera.position.z / 500, pyramidTextures);
                voxel.material.needsUpdate = true;
            });

            renderer.render(scene, camera);
        }

        // シーンの初期化
        init();

    </script>
</body>
</html>

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Interactive Voxel Rendering with Ray Casting and Shading</title>
    <style>
        body { margin: 0; overflow: hidden; }
        canvas { display: block; }
        .controls {
            position: absolute;
            top: 10px;
            left: 10px;
            z-index: 10;
            background: rgba(255, 255, 255, 0.8);
            padding: 10px;
        }
    </style>
</head>
<body>
    <!-- UIコントロール（スライダーとファイル入力） -->
    <div class="controls">
        <label>Gamma: <input type="range" id="gamma" min="0.1" max="3" value="1" step="0.1"></label><br>
        <label>Brightness: <input type="range" id="brightness" min="0" max="2" value="1" step="0.1"></label><br>
        <label>Red: <input type="range" id="red" min="0" max="2" value="1" step="0.1"></label><br>
        <label>Green: <input type="range" id="green" min="0" max="2" value="1" step="0.1"></label><br>
        <label>Blue: <input type="range" id="blue" min="0" max="2" value="1" step="0.1"></label><br>
        <input type="file" id="imageSelector" multiple accept="image/tif">
    </div>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
    <script>

        let scene, camera, renderer, voxelGroup, raycaster;
        let controls = {
            gamma: 1.0,
            brightness: 1.0,
            red: 1.0,
            green: 1.0,
            blue: 1.0
        };
        const imagePyramids = [];
        const pyramidLevels = 4;

        function init() {
            // シーンとカメラ、レンダラの初期化
            scene = new THREE.Scene();
            camera = new THREE.PerspectiveCamera(75, window.innerWidth / window.innerHeight, 0.1, 1000);
            camera.position.z = 500;

            renderer = new THREE.WebGLRenderer();
            renderer.setSize(window.innerWidth, window.innerHeight);
            document.body.appendChild(renderer.domElement);

            // レイキャスター
            raycaster = new THREE.Raycaster();

            // ボクセルグループ
            voxelGroup = new THREE.Group();
            scene.add(voxelGroup);

            // イベントリスナーを登録
            document.getElementById('gamma').addEventListener('input', updateControls);
            document.getElementById('brightness').addEventListener('input', updateControls);
            document.getElementById('red').addEventListener('input', updateControls);
            document.getElementById('green').addEventListener('input', updateControls);
            document.getElementById('blue').addEventListener('input', updateControls);
            document.getElementById('imageSelector').addEventListener('change', loadImages);

            animate();
        }

        // UIの値を更新
        function updateControls(event) {
            controls[event.target.id] = parseFloat(event.target.value);
            updateVoxelMaterials();
        }

        // ピラミッド画像の生成
        function createImagePyramid(baseImageUrl, levels) {
            const textures = [];
            for (let i = 0; i < levels; i++) {
                let scale = Math.pow(2, i);
                let url = `${baseImageUrl}_${scale}.tif`; // 低解像度バージョンをサーバーに用意しておく
                let texture = new THREE.TextureLoader().load(url);
                textures.push(texture);
            }
            return textures;
        }

        // ズームレベルに基づいてテクスチャを取得
        function getTextureForZoom(zoomLevel, pyramidTextures) {
            const level = Math.min(pyramidLevels - 1, Math.max(0, Math.floor(zoomLevel)));
            return pyramidTextures[level];
        }

        // 画像を読み込み、ボクセルを作成
        function loadImages(event) {
            const files = event.target.files;
            for (let i = 0; i < files.length; i++) {
                const file = files[i];
                const reader = new FileReader();
                reader.onload = function(e) {
                    const texture = new THREE.TextureLoader().load(e.target.result);
                    createVoxel(texture);
                };
                reader.readAsDataURL(file);
            }
        }

        // ボクセルの作成
        function createVoxel(texture) {
            const geometry = new THREE.BoxGeometry(1, 1, 1);
            const material = new THREE.MeshBasicMaterial({
                map: texture
            });
            const voxel = new THREE.Mesh(geometry, material);
            voxel.position.set(Math.random() * 500 - 250, Math.random() * 500 - 250, Math.random() * 500 - 250);
            voxelGroup.add(voxel);
        }

        // ボクセルのマテリアルを更新（ガンマ補正、RGB調整、輝度調整を適用）
        function updateVoxelMaterials() {
            voxelGroup.children.forEach(voxel => {
                voxel.material.map.encoding = THREE.sRGBEncoding; // sRGBエンコーディング
                voxel.material.map.color = new THREE.Color(
                    controls.red,
                    controls.green,
                    controls.blue
                );
                voxel.material.map.gammaFactor = controls.gamma;
                voxel.material.map.brightness = controls.brightness;
                voxel.material.needsUpdate = true;
            });
        }

        // レンダリングループ
        function animate() {
            requestAnimationFrame(animate);

            // ボクセルのテクスチャを更新
            voxelGroup.children.forEach((voxel, index) => {
                voxel.material.map = getTextureForZoom(camera.position.z / 500, imagePyramids[index]);
                voxel.material.needsUpdate = true;
            });

            renderer.render(scene, camera);
        }

        // シーンを初期化
        init();

    </script>
</body>
</html>

In [ ]:
import vtk
import numpy as np

def create_pyramid(images, levels):
    """Pyramid 이미지를 생성합니다."""
    pyramid_images = []
    for img in images:
        pyramid = []
        current = img
        for i in range(levels):
            pyramid.append(current)
            # 이미지 크기를 절반으로 줄입니다.
            current = vtk.vtkImageShrinkFilter()
            current.SetInputData(current)
            current.SetShrinkFactor(2)
            current.Update()
            current = current.GetOutput()
        pyramid_images.append(pyramid)
    return pyramid_images

def load_images(image_paths):
    """이미지를 로드하고 VTK 이미지 데이터로 변환합니다."""
    images = []
    for path in image_paths:
        reader = vtk.vtkTIFFReader()
        reader.SetFileName(path)
        reader.Update()
        images.append(reader.GetOutput())
    return images

def create_voxel_grid(images):
    """Voxel 그리드를 생성합니다."""
    width, height, depth = 20000, 20000, len(images)
    voxel_grid = vtk.vtkImageData()
    voxel_grid.SetDimensions(width, height, depth)
    voxel_grid.AllocateScalars(vtk.VTK_UNSIGNED_CHAR, 3)

    for z in range(depth):
        for y in range(height):
            for x in range(width):
                pixel = images[z].GetScalarComponentAsDouble(x, y, 0, 0)
                voxel_grid.SetScalarComponentFromDouble(x, y, z, 0, pixel)
                voxel_grid.SetScalarComponentFromDouble(x, y, z, 1, pixel)
                voxel_grid.SetScalarComponentFromDouble(x, y, z, 2, pixel)

    return voxel_grid

def render_voxel_grid(voxel_grid):
    """Voxel 그리드를 렌더링합니다."""
    volume_mapper = vtk.vtkSmartVolumeMapper()
    volume_mapper.SetInputData(voxel_grid)

    volume_property = vtk.vtkVolumeProperty()
    volume_property.ShadeOn()
    volume_property.SetInterpolationTypeToLinear()

    volume_actor = vtk.vtkVolume()
    volume_actor.SetMapper(volume_mapper)
    volume_actor.SetProperty(volume_property)

    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    renderer.AddActor(volume_actor)
    renderer.SetBackground(1, 1, 1)  # 배경 색상 설정

    # 마우스 인터랙션을 위한 설정
    style = vtk.vtkInteractorStyleTrackballCamera()
    render_window_interactor.SetInteractorStyle(style)

    render_window.Render()
    render_window_interactor.Start()

def main(image_paths):
    images = load_images(image_paths)
    pyramid_images = create_pyramid(images, levels=4)  # 피라미드 레벨 설정
    voxel_grid = create_voxel_grid(images)  # 최상위 이미지를 사용하여 voxel 그리드 생성
    render_voxel_grid(voxel_grid)

if __name__ == "__main__":
    # 이미지 경로 리스트 (실제 경로로 변경하세요)
    image_paths = ["image_1.tif", "image_2.tif", "...", "image_500.tif"]
    main(image_paths)

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>3D Voxel Ray Casting with Three.js</title>
    <style>
        body { margin: 0; }
        canvas { display: block; }
        #controls {
            position: absolute;
            top: 10px;
            left: 10px;
            background: rgba(255, 255, 255, 0.8);
            padding: 10px;
        }
    </style>
</head>
<body>
    <div id="controls">
        Brightness: <input type="range" id="brightness" min="0" max="2" step="0.1" value="1"><br>
        Gamma: <input type="range" id="gamma" min="0.1" max="3" step="0.1" value="1"><br>
        <input type="file" id="fileInput" multiple>
    </div>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
    <script>
        let scene, camera, renderer, raycaster, mouse, material, mesh;
        let voxelData = new Uint8Array(1000); // Placeholder for voxel data

        function init() {
            scene = new THREE.Scene();
            camera = new THREE.PerspectiveCamera(75, window.innerWidth / window.innerHeight, 0.1, 1000);
            renderer = new THREE.WebGLRenderer();
            renderer.setSize(window.innerWidth, window.innerHeight);
            document.body.appendChild(renderer.domElement);

            raycaster = new THREE.Raycaster();
            mouse = new THREE.Vector2();

            const geometry = new THREE.BoxGeometry(1, 1, 1);
            material = new THREE.MeshPhongMaterial({ color: 0xffffff, specular: 0x555555, shininess: 30 });
            mesh = new THREE.Mesh(geometry, material);
            scene.add(mesh);

            const light = new THREE.DirectionalLight(0xffffff, 1);
            light.position.set(1, 1, 1).normalize();
            scene.add(light);

            camera.position.z = 5;

            document.addEventListener('mousemove', onMouseMove, false);
            document.getElementById('brightness').addEventListener('input', updateBrightness, false);
            document.getElementById('gamma').addEventListener('input', updateGamma, false);
            document.getElementById('fileInput').addEventListener('change', handleFiles, false);

            animate();
        }

        function onMouseMove(event) {
            mouse.x = (event.clientX / window.innerWidth) * 2 - 1;
            mouse.y = -(event.clientY / window.innerHeight) * 2 + 1;
            raycaster.setFromCamera(mouse, camera);
            const intersects = raycaster.intersectObjects(scene.children);
            if (intersects.length > 0) {
                intersects[0].object.material.color.set(0xff0000);
            }
        }

        function updateBrightness(event) {
            material.color.setScalar(event.target.value);
        }

        function updateGamma(event) {
            renderer.gammaFactor = event.target.value;
        }

        function handleFiles(event) {
            const files = event.target.files;
            const loader = new THREE.TextureLoader();

            Array.from(files).forEach(file => {
                const url = URL.createObjectURL(file);
                loader.load(url, texture => {
                    const newMaterial = new THREE.MeshBasicMaterial({ map: texture });
                    const newMesh = new THREE.Mesh(new THREE.BoxGeometry(1, 1, 1), newMaterial);
                    scene.add(newMesh);
                });
            });
        }

        function animate() {
            requestAnimationFrame(animate);
            mesh.rotation.x += 0.01;
            mesh.rotation.y += 0.01;
            renderer.render(scene, camera);
        }

        window.addEventListener('resize', () => {
            camera.aspect = window.innerWidth / window.innerHeight;
            camera.updateProjectionMatrix();
            renderer.setSize(window.innerWidth, window.innerHeight);
        });

        init();
    </script>
</body>
</html>